## Problem 4: What is the closest shopping center from your home / work? (5 points)

In the last problem you should find out the closest shopping center from two different locations a) your home and b) work place. 

**Steps**:

 - Create a txt-file called `activity_locations.txt` (use same formatting as in Problem 1) with two columns:
    - `id`: unique id for each row
    - `addr`:  address of your work and home (or any two addresses in the Helsinki Region.)
    
Save the text file into this repository.
    
 - Read those addresses using pandas and geocode the addresses.
 - Find out the nearest shopping center to these points using Shapely `nearest_points`.
 - Print out the name of the shopping center that is nearest to a) home and b) work. For example: `Shopping center closest to home: REDI`.

In [1]:
# Import modules
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Read the data
fp = r'activity_locations.txt'
data = pd.read_csv(fp, sep=';')

data.head()

,id,addr
0,Senate,"00170 Helsinki, Finland"
1,Museum,"Mannerheimintie 34, 00100 Helsinki, Finland"


In [2]:
# import modules
from geopandas.tools import geocode

# Geocode activity locations
geo = geocode(data['addr'], provider='nominatim', user_agent='autogis_VU', timeout=4)
geo.head()

,geometry,address
0,POINT (24.95527 60.17196),"Kruununhaka, Eteläinen suurpiiri, Helsinki, He..."
1,POINT (24.93178 60.17496),"Kansallismuseo, 34, Mannerheimintie, Etu-Töölö..."


In [15]:
# existing data.crs (EPSG:4326), based on WGS84. Project layer to EPSG:3879 which is common to malls file, and is cartesian

# import required library
from pyproj import CRS

# perform data projection
geo = geo.to_crs(CRS.from_epsg(3879))

# join geo and data to keep malls id
geodata = geo.join(data)

# keep id and geometry cols
geodata = geodata[['id','geometry']]

# Define output filepath
out_fp = r'activity_locations.shp'

# save as shp file, in case it's needed some place else
geodata.to_file(out_fp)

In [28]:
# import libraries
from shapely.ops import nearest_points

# import shape file for malls
fp_path = r'shopping_centers.shp'
malls = gpd.read_file(fp_path)

# check crs is the same
# malls.crs == geodata.crs

# Find the closest shopping center for each activity location (closest shopping center from
# and closest shopping center from work) and print out the results:

def nearest_values(row, other_gdf, value_col='name'):
    
    # join all points into one object
    other_points = other_gdf['geometry'].unary_union
    
    # find geometry of the nearest point
    nearest_geom = nearest_points(row['geometry'], other_points)
    
    # find the actual row record by using a mask
    nearest_data = other_gdf[other_gdf['geometry'] == nearest_geom[1]]
    
    # find the cell value
    nearest_value = nearest_data[value_col]    
   
    return nearest_value.values[0]

geodata['nearest_loc'] = geodata.apply(nearest_values, other_gdf=malls, axis=1)
geodata


,id,geometry,nearest_loc
0,Senate,POINT (25497516.879 6673232.286),Forum
1,Museum,POINT (25496213.523 6673567.812),Forum


Can you think of other application cases for the nearest neighbour analysis?

In [ ]:
# REPLACE THE ERROR BELOW WITH YOUR OWN CODE
# raise NotImplementedError()